# From photo to piece


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from snap_fit.config.snap_fit_config import get_snap_fit_paths
from snap_fit.image.process import (
    find_contours,
    find_corners,
    find_sift_keypoints,
    find_white_regions,
)
from snap_fit.image.utils import (
    cut_rect_from_image,
    display_image,
    draw_contour,
    draw_contours,
    draw_corners,
    draw_keypoints,
    draw_regions,
    flip_colors_bw,
    load_image,
    pad_rect,
    save_image,
    show_image_mpl,
    sort_rects,
    translate_contour,
)
from snap_fit.puzzle.sheet import Sheet

In [ ]:
sf_paths = get_snap_fit_paths()
data_fol = sf_paths.data_fol
sample_fol = data_fol / "sample"

In [ ]:
# img_fn = "PXL_20241130_105107220.jpg"
# img_fn = "front_01.jpg"
# img_fn = "back_proc_02.jpg"
img_fn = "back_03.jpg"
# img_fn = "back_04.jpg"
# img_fn = "puzzle_pieces_01.jpeg"
img_fp = sample_fol / img_fn
img_fp

In [ ]:
sheet = Sheet(img_fp)

In [ ]:
show_image_mpl(sheet.img_orig)

In [ ]:
show_image_mpl(sheet.img_bw)

In [ ]:
for i, (x, y, w, h) in enumerate(sheet.regions[:10]):
    print(
        f"Black Region {i + 1}: x={x}, y={y}, width={w}, height={h}, area={w*h/1000:.2f}k"
    )

In [ ]:
annotated_image = draw_regions(sheet.img_orig, sheet.regions[:10])
show_image_mpl(annotated_image)

In [ ]:
for i, contour in enumerate(sheet.contours[:1]):
    print(f"Contour {i + 1}: {contour}")

In [ ]:
contour_image = draw_contours(sheet.img_orig, sheet.contours)

show_image_mpl(contour_image)

## grab a chunk of image


In [ ]:
region = sheet.regions[0]
region

In [ ]:
region_pad = pad_rect(region, 30, sheet.img_bw)
region_pad

In [ ]:
annotated_image = draw_regions(annotated_image, [region_pad])
show_image_mpl(annotated_image)

#### Use the Piece class


In [ ]:
piece = sheet.pieces[3]

In [ ]:
show_image_mpl(piece.img_bw)

In [ ]:
# show_image_mpl(piece.img_orig)
show_image_mpl(piece.img_gray)

### Find corners in the piece


In [ ]:
corners = find_corners(
    piece.img_gray,
    max_corners=30,
    quality_level=0.1,
    min_distance=40,
)
corners

In [ ]:
img_corners = draw_corners(piece.img_orig, corners)
show_image_mpl(img_corners)

#### Use SIFT and SURF


In [ ]:
# from snap_fit.image.process import find_surf_keypoints

In [ ]:
# keypoints, descriptors = find_surf_keypoints(piece.img_bw)
# keypoints, descriptors = find_sift_keypoints(piece.img_bw)
keypoints, descriptors = find_sift_keypoints(piece.img_gray)

In [ ]:
img_keypoints = draw_keypoints(piece.img_orig, keypoints)
show_image_mpl(img_keypoints)

## use contours


In [ ]:
# piece = sheet.pieces[0]

In [ ]:
contour = piece.contour
# contour

In [ ]:
x = -piece.region_pad[0]
y = -piece.region_pad[1]
print(f"Translating contour by {piece.region=} {x=}, {y=}")

c1 = translate_contour(contour, x, y)
# c1

In [ ]:
# draw the contour on the image

img_contour = draw_contour(piece.img_orig, c1)
# img_contour = draw_contour(piece.img_bw, c1, color=127)
show_image_mpl(img_contour)

In [ ]:
# draw the contour on the image

img_contour = draw_contour(piece.img_orig, piece.contour_loc)
# img_contour = draw_contour(piece.img_bw, c1, color=127)
show_image_mpl(img_contour)

### Derivative of a contour


In [ ]:
piece = sheet.pieces[0]

In [ ]:
cl = piece.contour_loc

In [ ]:
cl.shape

In [ ]:
cl[:10, 0]

In [ ]:
cl[-10:, 0]

In [ ]:
from snap_fit.image.contour import Contour


cc = Contour(cl)
cc.derive(step=5)
print(cc.derivative.shape)
cc.derivative[:, 0]

In [ ]:
# cl[3][0]
# cc.derivative[3][0]

In [ ]:
from snap_fit.image.utils import draw_contour_derivative


cont_d_image = draw_contour_derivative(
    piece.img_orig,
    cl,
    cc.derivative,
    skip=1,
    arrow_length=20,
)
show_image_mpl(cont_d_image)

In [ ]:
# compute the orientation of the gradient

d = cc.derivative
d[:, 0]

In [ ]:
import numpy as np


ori = np.arctan2(d[:, 0, 1], d[:, 0, 0])
# the orientation can be between -pi and pi
# we can take the modulo to get the orientation between 0 and 2*pi
ori = np.mod(ori, 2 * np.pi)
ori[:10]

In [ ]:
ori_unwrapped = np.unwrap(ori)
print(ori_unwrapped[:10])
print(ori_unwrapped[-10:])

In [ ]:
# compute the magnitude of the gradient
mag = np.linalg.norm(d, axis=2)[:, 0]
mag[:10]

In [ ]:
def derive(
    data: np.ndarray,
    step: int = 5,
) -> np.ndarray:
    """Derives the contour to get the orientation and curvature.

    For each point on the contour, the derivative is calculated using the central difference method.
    The step size determines the distance between the points used for the derivative.

    Args:
        step (int): The step size for the derivative (default is 5).
    """
    # as the contour is a closed curve, we can calculate the derivative by
    # wrapping around the end points to the start points
    c_wrap = np.hstack((data, data[:step]))
    # also wrap around the start points to the end points
    c_wrap = np.hstack((data[-step:], c_wrap))
    print(c_wrap[:10])
    print(c_wrap[-10:])

    # unwrap the angles to avoid the discontinuity at 2*pi
    c_wrap_unwrap = np.unwrap(c_wrap)

    # Calculate the derivative of the contour
    d_wrap = np.gradient(c_wrap_unwrap, step, axis=0)

    # Calculate the derivative of the contour
    # d_wrap = np.diff(c_wrap_unwrap, axis=0)

    # d_wrap = data[step:] + 2 * np.pi - data[:-step]
    # print(d_wrap[:10])
    # d_wrap = np.mod(d_wrap, 2 * np.pi)
    # print(d_wrap[:10])

    # Remove the wrapped points
    d = d_wrap[step:-step]
    return d

In [ ]:
def smooth(
    data: np.ndarray, window_len: int = 11, window: str = "hanning"
) -> np.ndarray:
    """Smooth the data using a window with requested size.

    Args:
        data (np.ndarray): The data to be smoothed.
        window_len (int): The size of the window (default is 11).
        window (str): The type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman' (default is 'hanning').

    Returns:
        np.ndarray: The smoothed data.
    """
    if data.ndim != 1:
        raise ValueError("smooth only accepts 1 dimension arrays.")

    if data.size < window_len:
        raise ValueError("Input vector needs to be bigger than window size.")

    if window_len < 3:
        return data

    if not window in ["flat", "hanning", "hamming", "bartlett", "blackman"]:
        raise ValueError(
            f"Window is not one of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'"
        )

    # s = np.r_[data[window_len - 1 : 0 : -1], data, data[-2 : -window_len - 1 : -1]]
    s = np.r_[data[window_len - 1 : 0 : -1], data, data[0 : -window_len - 1 : -1]]

    # # as the contour is a closed curve, we can wrap around
    # s = np.hstack((data, data[: window_len + 1]))
    # s = np.hstack((data[-(window_len + 1) :], s))

    if window == "flat":  # moving average
        w = np.ones(window_len, "d")
    else:
        w = eval(f"np.{window}({window_len})")

    y = np.convolve(w / w.sum(), s, mode="valid")
    print(f"{data.shape=}, {s.shape=} {y.shape=}")
    return y  # [window_len + 2 : -(window_len + 0)]

In [ ]:
ori_unwrapped_smooth = smooth(ori_unwrapped, window_len=5, window="flat")
# ori_unwrapped_smooth = smooth(ori_unwrapped, window_len=5, window="hanning")
print(ori_unwrapped_smooth.shape)
print(ori_unwrapped_smooth[:10])
print(ori_unwrapped_smooth[-10:])

In [ ]:
# ori_d = derive(ori, step=5)
ori_d = derive(ori_unwrapped_smooth, step=5)
# ori_d = np.mod(ori_d, np.pi)
# ori_d[:10]
ori_d[115:120]

In [ ]:
ori[115:120]

In [ ]:
ori[115:120] - 2 * np.pi

In [ ]:
# plot the orientation and magnitude of the gradient
from matplotlib import pyplot as plt

fig, ax = plt.subplots(4, 1, figsize=(12, 12))
min_idx = 326
max_idx = 330
# also add points for the derivative
ax[0].scatter(range(max_idx - min_idx), ori[min_idx:max_idx], color="red")
ax[0].plot(ori[min_idx:max_idx])
ax[0].set_title("Orientation of the Gradient")
ax[1].plot(ori_unwrapped[min_idx:max_idx])
ax[1].set_title("Unwrapped Orientation of the Gradient")
ax[2].plot(ori_d[min_idx:max_idx])
ax[2].set_title("Derivative of the Orientation")
ax[3].plot(mag[min_idx:max_idx])
ax[3].set_title("Magnitude of the Gradient")
plt.show()

print(ori[min_idx:max_idx])
print(ori_d[min_idx:max_idx])

In [ ]:
# plot the orientation and magnitude of the gradient
from matplotlib import pyplot as plt

fig, ax = plt.subplots(4, 1, figsize=(12, 13))
# also add points for the derivative
# ax[0].plot(ori)
# ax[0].set_title("Orientation of the Gradient")
ax[0].plot(ori_unwrapped)
ax[0].set_title("Unwrapped Orientation of the Gradient")
ax[0].grid()
ax[1].plot(ori_unwrapped_smooth / np.pi * 180)
ax[1].set_title("Smoothed Unwrapped Orientation of the Gradient")
ax[1].grid()
ax[2].plot(ori_d)
ax[2].set_title("Derivative of the Orientation")
ax[2].grid()
ax[3].plot(mag)
ax[3].set_title("Magnitude of the Gradient")
plt.show()

### incredibly naive approach


In [ ]:
piece = sheet.pieces[5]

In [ ]:
shap = piece.img_bw.shape
print(shap)

In [ ]:
piece.img_bw.dtype

In [ ]:
# import cv2

# from snap_fit.image.utils import draw_line


# diag_mask = np.zeros(shap, dtype=np.uint8)
# # thick = 150
# thick = int(sum(shap) / 2 / 4 * 1.05)
# print(thick)
# # diag_mask = cv2.line(diag_mask, (0, 0), (shap[1], shap[0]), 255, thick)
# # diag_mask = cv2.line(diag_mask, (0, shap[0]), (shap[1], 0), 255, thick)
# # show_image_mpl(diag_mask)
# diag_mask = draw_line(diag_mask, (0, 0), (shap[1], shap[0]), 255, thick)
# diag_mask = draw_line(diag_mask, (0, shap[0]), (shap[1], 0), 255, thick)

In [ ]:
img_crossed = piece.img_bw // 2 + piece.cross_mask // 2
show_image_mpl(img_crossed)

In [ ]:
# img_crossmasked = cv2.bitwise_and(piece.img_bw, diag_mask)
show_image_mpl(piece.img_crossmasked)

In [ ]:
# # sweep the image with a diagonal line starting from the corner
# # and stop when the line hits the crossmasked image
# # this will give the corner of the piece


# def find_corner(
#     img_crossmasked: np.ndarray,
#     which_corner: str,
# ) -> tuple:
#     """Find the corner of the piece by sweeping the image.

#     The function sweeps the image with a line starting from the corner,
#     orthogonal to the diagonal of the image, and stops when the line hits the
#     crossmasked image.
#     The corner is then the point where the line hits the crossmasked image.

#     Args:
#         img_crossmasked (np.ndarray): The image with the diagonal line.
#         which_corner (str): The corner to find, one of
#             "top_left", "top_right", "bottom_left", "bottom_right".

#     Returns:
#         tuple: The coordinates of the corner, as a tuple (x, y).
#     """
#     shap = img_crossmasked.shape
#     for i in range(min(shap)):
#         for j in range(i):
#             match which_corner:
#                 case "top_left":
#                     x = j
#                     y = i - j
#                 case "bottom_left":
#                     x = i - j
#                     y = shap[0] - j - 1
#                 case "top_right":
#                     x = shap[1] - i + j
#                     y = j
#                 case "bottom_right":
#                     x = shap[1] - j - 1
#                     y = shap[0] - i + j
#                 case _:
#                     raise ValueError(f"Invalid corner {which_corner=}")
#             if img_crossmasked[y, x] > 0:
#                 return (x, y)
#     return (0, 0)


# # corner = find_corner(img_crossmasked, "top_left")
# # corners = [corner]

# corners = []
# for corner in [
#     "top_left",
#     "top_right",
#     "bottom_left",
#     "bottom_right",
# ]:
#     corners.append(find_corner(img_crossmasked, corner))
# corners

In [ ]:
# img_corners = draw_corners(piece.img_orig, [corner])
img_corners = draw_corners(
    # piece.img_orig,
    img_crossed,
    [
        # (10, 10),
        # (20, 40),
        # (shap[1] - 10, shap[0] - 10),
        # (shap[1] - 10, 10),
        # (10, shap[0] - 10),
        # (min(shap) - 10, min(shap) - 10),
        *piece.corners.values(),
    ],
    color=190,
)
show_image_mpl(img_corners)